# Module 04: Linear Regression 

## Prerequisites

In [20]:
# Helper packages
import pandas as pd
import numpy as np

# Modeling packages
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import compose
from sklearn import cross_decomposition
from sklearn import decomposition
from sklearn import model_selection
from sklearn import linear_model
from sklearn import pipeline

In [21]:
# Ames housing data
ames = pd.read_csv("../../00-data/ML/ames.csv")

# create train/test split
train, test = train_test_split(ames, train_size=0.7, random_state=123)

# separate features from labels and only use numeric features
X_train = train.drop("Sale_Price", axis=1)
y_train = train[["Sale_Price"]]

## Simple linear regression

### Best fit line

In [22]:
# create linear regression model object
lm_mod = linear_model.LinearRegression()

# fit linear model with only Gr_Liv_Area feature
lm_fit = lm_mod.fit(X_train[["Gr_Liv_Area"]], y_train)

### Coefficients

In [23]:
# intercept
lm_fit.intercept_

array([15770.15274496])

In [24]:
# coefficient for Gr_Liv_Area
lm_fit.coef_

array([[110.54517166]])

## Multiple linear regression

In [25]:
# create linear regression model object
lm_mod = linear_model.LinearRegression()

# fit linear model with only Gr_Liv_Area and Year_Built feature
lm_fit = lm_mod.fit(X_train[["Gr_Liv_Area", "Year_Built"]], y_train)

### Coefficients

In [26]:
# intercept
lm_fit.intercept_

array([-2127545.21682598])

In [27]:
# coefficients for Gr_Liv_Area and Year_Built
lm_fit.coef_

array([[  94.82771439, 1099.10620648]])

### Interactions

In [28]:
# create linear regression model object
lm_mod = linear_model.LinearRegression()

# use PolynomialFeatures to create main Gr_Liv_Area and Year_Built effects and
# also an interaction effect between Gr_Liv_Area & Year_Built
effects = preprocessing.PolynomialFeatures(
  interaction_only=True,
  include_bias=False
  )
features = effects.fit_transform(X_train[["Gr_Liv_Area", "Year_Built"]])

# fit linear model with only Gr_Liv_Area and Year_Built feature and
# also include an interaction effect (Gr_Liv_Area:Year_Built)
lm_fit = lm_mod.fit(features, y_train)

In [29]:
# coefficients for Gr_Liv_Area, Year_Built effects and the interaction 
# effect between Gr_Liv_Area & Year_Built
lm_fit.coef_

array([[-6.62106855e+02,  4.90386820e+02,  3.84056493e-01]])

### Many features

In [30]:
# create new feature set with categorical features dummy encoded
encoder = preprocessing.OneHotEncoder(drop='first')
cat_feat_only = compose.make_column_selector(dtype_include="object")
preprocessor = compose.ColumnTransformer(
  remainder="passthrough",
  transformers=[("one-hot", encoder, cat_feat_only)]
  )
X_train_encoded = preprocessor.fit_transform(X_train)

# MLR model with new dummy encoded feature set
lm_mod = linear_model.LinearRegression()
lm_fit = lm_mod.fit(X_train_encoded, y_train)

In [31]:
# first 10 coefficients
lm_fit.coef_[0, 0:10]

array([  6916.78584629,   2479.30422373, -12417.78047706,  14002.25524953,
        16540.4309626 ,  19940.43517711,  -6655.64145886,  -4857.84346031,
        -5358.67224573,  -1201.17472585])

## Assessing model accuracy

In [32]:
# feature sets to compare across
feature_set1 = X_train[["Gr_Liv_Area"]]
feature_set2 = X_train[["Gr_Liv_Area", "Year_Built"]]
feature_set3 = X_train_encoded
feature_sets = {'lm1': feature_set1, 'lm2': feature_set2, 'lm3': feature_set3}

# define loss function
loss = 'neg_root_mean_squared_error'

# create 10 fold CV object
kfold = model_selection.KFold(n_splits=10, random_state=8451, shuffle=True)

# object to store CV RMSE results
results = {}

for name, feat in feature_sets.items():
  # create LM model object
  lm_mod = linear_model.LinearRegression()

  # execute and score the cross validation procedure
  cv_results = model_selection.cross_val_score(
    estimator=lm_mod, 
    X=feat, 
    y=y_train, 
    cv=kfold, 
    scoring=loss
    )
  results[name] = np.absolute(cv_results.mean())

In [33]:
results

{'lm1': 58191.00896279409,
 'lm2': 48202.332097374696,
 'lm3': 34585.102640984886}

## Principal component regression

In [34]:
# create linear model object
lm_mod = linear_model.LinearRegression()

# create k-fold cross validation object
kfold = model_selection.KFold(n_splits=5, random_state=8451, shuffle=True)

# define loss function
loss = 'neg_root_mean_squared_error'

# create our preprocessing steps which includes performing PCA 
# with 10 components
scaler = preprocessing.StandardScaler()
pca = decomposition.PCA(n_components=10)
encoder = preprocessing.OneHotEncoder(handle_unknown="ignore")
num_feat_only = compose.make_column_selector(dtype_include="number")
cat_feat_only = compose.make_column_selector(dtype_include="object")

# combine all steps into a pre-processing pipeline
preprocessor = compose.ColumnTransformer(
  remainder="passthrough",
  transformers=[
  ("std_encode", scaler, num_feat_only),
  ("pca_encode", pca, num_feat_only),
  ("one-hot", encoder, cat_feat_only),
  ])

# create a pipeline object that combines model with recipe
model_pipeline = pipeline.Pipeline(steps=[
  ("preprocessor", preprocessor),
  ("lm", lm_mod),
])

# train and fit our model
cv_results = model_selection.cross_val_score(
  estimator=model_pipeline, 
  X=X_train, 
  y=y_train, 
  cv=kfold, 
  scoring=loss
  )

# get results
np.absolute(cv_results.mean())

34240.32920593013

### Tuning

In [35]:
# create k-fold cross validation object
kfold = model_selection.KFold(n_splits=5, random_state=8451, shuffle=True)

# define loss function
loss = 'neg_root_mean_squared_error'

# create our preprocessing steps
scaler = preprocessing.StandardScaler()
encoder = preprocessing.OneHotEncoder(handle_unknown="ignore")
num_feat_only = compose.make_column_selector(dtype_include="number")
cat_feat_only = compose.make_column_selector(dtype_include="object")

# create object to save results
results = {}

# iterate over over 2, 4, 6, ..., 26 components and train model
for n_comp in range(2, 28, 2):
  # create PCA object with n components
  pca = decomposition.PCA(n_components=n_comp)
  
# combine all steps into a pre-processing pipeline
  preprocessor = compose.ColumnTransformer(
    remainder="passthrough",
    transformers=[
    ("std_encode", scaler, num_feat_only),
    ("pca_encode", pca, num_feat_only),
    ("one-hot", encoder, cat_feat_only),
    ])

  # create linear model object
  lm_mod = linear_model.LinearRegression()

  # create a pipeline object that combines model with recipe
  model_pipeline = pipeline.Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("lm", lm_mod),
  ])

  # train and fit our model
  cv_results = model_selection.cross_val_score(
    estimator=model_pipeline, 
    X=X_train, 
    y=y_train, 
    cv=kfold, 
    scoring=loss
    )

  # get results
  results[n_comp] = np.absolute(cv_results.mean())

In [36]:
pd.DataFrame.from_dict(
  results,
  orient='index',
  columns=['RMSE']
  ).rename_axis('n_components').reset_index()

,n_components,RMSE
0,2,35615.069307
1,4,34680.852289
2,6,34286.038419
3,8,34403.641118
4,10,34249.517052
5,12,34124.535235
6,14,34117.588820
7,16,33973.949462
8,18,33785.814554
9,20,33415.438436


## Partial least squares

In [37]:
# create linear model object
pls_mod = cross_decomposition.PLSRegression()

# create k-fold cross validation object
kfold = model_selection.KFold(n_splits=5, random_state=8451, shuffle=True)

# define loss function
loss = 'neg_root_mean_squared_error'

# create our preprocessing steps to normalize and one-hot encode
scaler = preprocessing.StandardScaler()
encoder = preprocessing.OneHotEncoder(handle_unknown="ignore", sparse=False)
num_feat_only = compose.make_column_selector(dtype_include="number")
cat_feat_only = compose.make_column_selector(dtype_include="object")

# combine all steps into a pre-processing pipeline
preprocessor = compose.ColumnTransformer(
  remainder="passthrough",
  transformers=[
    ("std_encode", scaler, num_feat_only),
    ("one-hot", encoder, cat_feat_only),
  ])

# create a pipeline object that combines model with recipe
model_pipeline = pipeline.Pipeline(steps=[
  ("preprocessor", preprocessor),
  ("pls", pls_mod),
])

# Create grid of hyperparameter values
hyper_grid = {'pls__n_components': range(2, 28, 2)}

# Tune a knn model using grid search
grid_search = model_selection.GridSearchCV(
  model_pipeline, 
  hyper_grid, 
  cv=kfold, 
  scoring=loss
  )
  
results = grid_search.fit(X_train, y_train)

# Best model's cross validated RMSE
abs(results.best_score_)

30834.84469515183

In [38]:
# Optimal number of components
results.best_estimator_.get_params().get('pls__n_components')

4

## Exercises

Using the Boston housing data set where the response feature is the median value of homes within a census tract (`cmedv`):

1. Pick a single feature and apply a simple linear regression model.
   - Interpret the feature's coefficient
   - What is the model's performance? How does it compare to the KNN in the last module?
2. Pick another feature to add to the model.
   - Before applying the model why do you think this feature will help?
   - Apply a linear regression model with the two features and compare to the simple linear model.
   - Interpret the coefficients.
3. Now apply a model that includes all the predictors.
   - How does this model compare to the previous two?
4. Can you identify any model concerns?
5. Apply a principal component regression model.
   - Perform a grid search over several components.
   - Identify and explain the performance of the optimal model.
6. Now apply a partial least square model.
   - Perform a grid search over several components.
   - Identify and explain the performance of the optimal model.